# Image alignment

The purpose of this notebook is to resize and align the ppl, xpl, and false-color TIMA images for use in data collection.


In [ ]:
# import libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import imutils

# Test out manual pixel coordinates

In [ ]:
# create two lists of matched coordinate pairs

#xpl
pts1 = np.array([[126, 124], [144, 153], [188, 175], [83, 120], [10,49],
                 [138, 71],[52, 168], [28, 182], [124, 25], [79,20]])

#ppl
pts2 = np.array([[125, 74], [152, 56], [175, 11], [120, 116], [50, 190],
                 [71, 61],[168, 147], [182, 170], [25, 77], [19, 120]])
# load and view sample image
img = cv2.imread('Images/xplimg.png')
template = cv2.imread('Images/template.jpg')


plt.figure(figsize=(10,15))
plt.subplot(1,2,1)
plt.imshow(template)
# plot dots
plt.scatter(pts2[:,0], pts2[:,1], color='r')
plt.title('Template Image')

plt.subplot(1,2,2)
plt.imshow(img)
# plot red dot
plt.scatter(pts1[:,0], pts1[:,1],color='r')
plt.title('XPL Image')


In [ ]:
(H, mask) = cv2.findHomography(pts1, pts2, method = cv2.RANSAC)

# use the homography matrix to align the images
(h, w) = template.shape[:2]
aligned = cv2.warpPerspective(img, H, (w,h))

# resize both the aligned and template images so we can easily 
# visualize them on the screen
aligned = imutils.resize(aligned, width=700)
template = imutils.resize(template, width=700)

# side-by-side comparison of the output aligned image and the template
stacked = np.hstack([aligned, template])

# second image alignment visualization will be overlaying the
# aligned image on the template to get an idea of how good
# the image alignment is

overlay = template.copy()
output = aligned.copy()
cv2.addWeighted(overlay, 0.5, output, 0.5, 0, output)

# show the two output inmage alignment visualizations
plt.figure(figsize=(10,10))

plt.subplot(2,1,1)
plt.title('Stacked image')
plt.imshow(stacked)
plt.subplot(2,1,2)
plt.title('Overlayed image')
plt.imshow(output)

plt.tight_layout()

# Develop way to georeference both images and manually select keypoints

In [ ]:
template = Image.open('Images/xpltest.png')
template.size

In [1]:
# import libraries
import cv2
import numpy as np
from PIL import Image, ImageTk
import imutils
import tkinter as tk
# import matplot lib like this, otherwise there is some error with tkinter
import matplotlib
matplotlib.use("TkAgg")
from matplotlib import pyplot as plt

# lists to hold linked pixel coordinates
ppl_coords = []
xpl_coords = []
label_coords = []

FRAME_WIDTH = 500
FRAME_HEIGHT = 800

ppl = Image.open('/Users/kacikus/Dropbox/AutomatedMineralogy_Project/Images/ppltest.jpg')
xpl = Image.open('/Users/kacikus/Dropbox/AutomatedMineralogy_Project/Images/xpltest.png')
labels = Image.open('/Users/kacikus/Dropbox/AutomatedMineralogy_Project/Images/labels.png')

# get  dimensions of the img/template photos
width_ppl, height_ppl = ppl.size
width_xpl, height_xpl = xpl.size
width_lab, height_lab = xpl.size

# since we are resizing the images to have a height of FRAME_WIDTH px and width relative to that height
# we need an x and y scaler to adjust the pixel locations
scaler_ppl = height_ppl/800
scaler_xpl = height_xpl/800
scaler_lab = height_lab/800

# gets the pixel locations from ppl imate
def getxy_ppl(event):
    print("ppl pxl location = ({0},{1})".format(int(event.x*scaler_ppl), int(event.y*scaler_ppl)))
    ppl_coords.append([int(event.x*scaler_ppl), int(event.y*scaler_ppl)])
    id = canvas1.create_oval(event.x-4, event.y-4, event.x+4, event.y+4, fill='yellow')

# gets pixel locations from the xpl image
def getxy_xpl(event):
    print("xpl pxl location = ({0},{1})".format(int(event.x*scaler_xpl), int(event.y*scaler_xpl)))
    xpl_coords.append([int(event.x*scaler_xpl), int(event.y*scaler_xpl)])
    id = canvas2.create_oval(event.x-4, event.y-4, event.x+4, event.y+4, fill='yellow')

# gets pixel locations from the false color image
def getxy_lab(event):
    print("label pxl location = ({0},{1})".format(int(event.x*scaler_lab), int(event.y*scaler_lab)))
    label_coords.append([int(event.x*scaler_lab), int(event.y*scaler_lab)])
    id = canvas3.create_oval(event.x-4, event.y-4, event.x+4, event.y+4, fill='yellow')

   
# zoom in and out of the image
#def zoom(event=None):

# root window for the application
root = tk.Tk(className='georef window')

# assign
root.config(cursor="draft_small")

# make dimensions of main window fit the two images
root.geometry("{0}x{1}".format(FRAME_HEIGHT*3, FRAME_HEIGHT))

# create first frame
frame1 = tk.Frame(root, width= FRAME_WIDTH, height=FRAME_HEIGHT)
frame1.grid(row=0, column=0, padx=0, pady=0)
# create second frame
frame2 = tk.Frame(root, bg='#c4ffd2', width= FRAME_WIDTH, height=FRAME_HEIGHT)
frame2.grid(row=0, column=1, padx=0, pady=0)
# create third frame
frame3 = tk.Frame(root, bg='pink', width= FRAME_WIDTH, height=FRAME_HEIGHT)
frame3.grid(row=0, column=2, padx=0, pady=0)


# create canvas for ppl
canvas1 = tk.Canvas(frame1, bg='green', width=FRAME_WIDTH, height=FRAME_HEIGHT)
canvas1.grid(row=0, column=0, sticky='nesw')
# create canvas for xpl
canvas2 = tk.Canvas(frame2, bg='blue', width=FRAME_WIDTH, height=FRAME_HEIGHT)
canvas2.grid(row=0, column=1, sticky='nesw')
# create canvas for labels
canvas3 = tk.Canvas(frame3, bg='pink', width=FRAME_WIDTH, height=FRAME_HEIGHT)
canvas3.grid(row=0, column=2, sticky='nesw')

# create image location resize it (width, height)
canvas1.image = ImageTk.PhotoImage(ppl.resize((int(np.ceil(width_ppl/scaler_ppl)), FRAME_HEIGHT), Image.ANTIALIAS))
canvas1.create_image(0, 0, image=canvas1.image, anchor='nw')

canvas2.image = ImageTk.PhotoImage(xpl.resize((int(np.ceil(width_xpl/scaler_xpl)), FRAME_HEIGHT), Image.ANTIALIAS))
canvas2.create_image(0, 1, image=canvas2.image, anchor='nw')

canvas3.image = ImageTk.PhotoImage(labels.resize((int(np.ceil(width_xpl/scaler_lab)), FRAME_HEIGHT), Image.ANTIALIAS))
canvas3.create_image(0, 2, image=canvas3.image, anchor='nw')

canvas1.bind('<Button-1>', getxy_ppl)
canvas2.bind('<Button-1>', getxy_xpl)
canvas3.bind('<Button-1>', getxy_lab)


# runs the application 
root.mainloop()

xpl pxl location = (412,966)
xpl pxl location = (628,1007)
xpl pxl location = (562,682)


In [ ]:
np.max([2, 3])

In [ ]:
img_coords

### Reference:
https://www.programmersought.com/article/38705290853/

# Method 2
The above method did not work very well and warped the image quite a bit.

I need to:
1. Transform an image to fit perfectly on top on another image
2. crop so they have the same pixel positions

In [ ]:
img = cv2.imread('Images/ppltest.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
kernel = np.ones((25,25), np.float32)/625
avg = cv2.filter2D(img, -1, kernel)
blur = cv2.GaussianBlur(img, (25,25),0)

plt.figure(figsize=(15,20))
plt.subplot(131),plt.imshow(img),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(dst),plt.title('Averaging')
plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(blur),plt.title('Gaussian Blur')
plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
kernel.size

# Edge detection

In [ ]:
img = cv2.imread('Images/image.jpg')
edges = cv2.Canny(img, 100, 100)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

In [ ]:
img = cv2.imread('Images/xplimg.png')
edges = cv2.Canny(img, 50, 2)

plt.subplot(121),plt.imshow(img,cmap = 'gray')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

# compare image matches

In [ ]:
# open images
# transformed ppl image
#img = cv2.imread('Images/image.jpg')

# larger ppl image
#img = cv2.imread('Images/larger_img.jpg')

# transformed xpl image
img = cv2.imread('Images/xplimg.png')
imgGray = cv2.Canny(img, 50, 2)

# original ppl image
template = cv2.imread('Images/template.jpg')
templateGray = cv2.Canny(template, 100, 100)

'''Uncomment when you arent using edges'''
# img = cv2.imread('smileyface.jpeg')
# template = cv2.imread('template.jpeg')

# convert to grayscale
#imageGray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#templateGray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
height, width = templateGray.shape

# use ORB to detect keypoints and extract (binary) 
# local invariant features
orb = cv2.ORB_create(500)
kp1, d1 = orb.detectAndCompute(imageGray, None)
kp2, d2 = orb.detectAndCompute(templateGray, None)

# match features
method = cv2.DESCRIPTOR_MATCHER_BRUTEFORCE_HAMMING
matcher = cv2.DescriptorMatcher_create(method)
matches = matcher.match(d1, d2, None)

# sort matches by their distance (smaller distance,
# means more similar features)
matches = sorted(matches, key=lambda x: x.distance)
no_of_matches = len(matches)

# keep only the top matches
keep = int(len(matches) * 0.05)
matches = matches[:keep]

# check to see if we should visualize the matched keypoints
matchedVis = cv2.drawMatches(img, kp1, template, kp2, matches, None)
matchedVis = imutils.resize(matchedVis, width = 1000)

plt.figure(figsize=(10,8))
plt.imshow(matchedVis)

In [ ]:
# allocate memory for the keypoints (x, y)-coordinates from the
# top matches -- we'll use these coordinates to compute homography matrix
pts1 = np.zeros((len(matches), 2), dtype='float')
pts2 = np.zeros((len(matches), 2), dtype='float')

# loop over the top matches
for (i, m) in enumerate(matches):
    # indicate that the two keypoints int he respective images
    # map to each other
    pts1[i] = kp1[m.queryIdx].pt
    pts2[i] = kp2[m.trainIdx].pt

Given our organized pairs of keypoint matches, now we are ready to align the image

In [ ]:
# compute homography matrix between two sets of matched points
(H, mask) = cv2.findHomography(pts1, pts2, method = cv2.RANSAC)

# use the homography matrix to align the images
(h, w) = template.shape[:2]
aligned = cv2.warpPerspective(img, H, (w,h))


plt.figure(figsize=(8,10))
# plot original (left) and template (right)
plt.subplot(1,2,1)
plt.title('Original (left); template (right)')
plt.imshow(matchedVis)
# plot the newly aligned image
plt.subplot(1,2,2)
plt.title('Aligned image')
plt.imshow(aligned)

# compare both the newly aligned image and the old image stacked on top of each other

In [ ]:
# resize both the aligned and template images so we can easily 
# visualize them on the screen
aligned = imutils.resize(aligned, width=700)
template = imutils.resize(template, width=700)

# side-by-side comparison of the output aligned image and the template
stacked = np.hstack([aligned, template])

# second image alignment visualization will be overlaying the
# aligned image on the template to get an idea of how good
# the image alignment is

overlay = template.copy()
output = aligned.copy()
cv2.addWeighted(overlay, 0.5, output, 0.5, 0, output)

# show the two output inmage alignment visualizations
plt.figure(figsize=(10,10))

plt.subplot(2,1,1)
plt.title('Stacked image')
plt.imshow(stacked)
plt.subplot(2,1,2)
plt.title('Overlayed image')
plt.imshow(output)

plt.tight_layout()

## References

https://www.geeksforgeeks.org/image-registration-using-opencv-python/